# Colombian Constitution EDA.
    Brief resume about the methodology and the results that we got.

## About the Document
    The Political Constitution of the Republic of Colombia of 1991 is the Magna Carta of the Republic of Colombia. It was promulgated in the Constitutional Gazette number 114 of Thursday, July 4, 1991, and is also known as the Constitution of Human Rights. It replaced the Political Constitution of 1886 and was issued during the presidency of the liberal César Gaviria.1

## DS Pipeline



## Document Data Model

PowerPoint graph, meanwhile we can use the hierarchy of the articles dictionary

```
hierarchy = {
    'TITULO' : 'h1',
    'DISPOSICIONES' : 'h1',
    'CAPITULO' : 'h2',
    'ARTÍCULO' : 'p'

}
```

## EDA

### Libraries 

In [1]:
import numpy as np
import pandas as pd
# For visualizations
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
# import tensorflow  as tf
# # tf.disable_v2_behavior()
# import tensorflow_hub as hub
# import tensorflow_text
from os import path
from tf_model import embed_text


In [2]:
root_folder = "LegalSearcher/ElasticSearch"
constitution_f = f'{root_folder}/embedding.json'
filepath = path.abspath(path.join('','..', '..', constitution_f))
#import Dataset
df = pd.read_json(filepath)

In [4]:
df=df[['id','embedding']]
df.replace(r'\n','', regex=True,inplace=True) 
df['embedding'] = df['embedding'].apply(lambda x:x[1:])
df.rename(columns={'embedding':'article_list'},inplace=True)
df.head()


,id,article_list
0,const01000001,"[Colombia es un Estado social de derecho, orga..."
1,const01000002,[Son fines esenciales del Estado: servir a la ...
2,const01000003,[La soberanía reside exclusivamente en el pueb...
3,const01000004,"[La Constitución es norma de normas., En todo ..."
4,const01000005,"[El Estado reconoce, sin discriminación alguna..."


In [6]:
###### Sentence Encoder for DataFrame
df['embedds'] = df['article_list'].apply(lambda x:embed_text(x))
df.head()
# art1_res = embed(art1)

,id,article_list,embedds
0,const01000001,"[Colombia es un Estado social de derecho, orga...","[0.024026740342378616, -0.07912702113389969, 0..."
1,const01000002,[Son fines esenciales del Estado: servir a la ...,"[0.05873824283480644, 0.07823819667100906, -0...."
2,const01000003,[La soberanía reside exclusivamente en el pueb...,"[-0.003973131999373436, -0.007816539146006107,..."
3,const01000004,"[La Constitución es norma de normas., En todo ...","[0.06180395931005478, -0.022052446380257607, 0..."
4,const01000005,"[El Estado reconoce, sin discriminación alguna...","[0.027791915461421013, -0.018517587333917618, ..."


In [7]:
query0 = embed_text(['Colombia es un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista, fundada en el respeto de la dignidad humana, en el trabajo y la solidaridad de las personas que la integran y en la prevalencia del interés general'])
query1 = embed_text(['organizacion de colombia'])
query2 = embed_text(['Fines esenciales del estado'])
query3 = embed_text(['Representantes del pueblo'])
query4 = embed_text(['derechos humanos'])
query5 = embed_text(['tratados y convenios'])
query6 = embed_text(['Lorem ipsum dolor sit amet'])
query7 = embed_text(['organos del poder publico'])
query8 = embed_text(['Propiedad Intelectual'])
query9 = embed_text(['cuidado del medio ambiente'])
query10 = embed_text(['Ley de 1992'])
query11 = embed_text(['pizza'])

In [8]:
# Cosine Similarity Function
def similarity(A,B):
    cos_sim = np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))+1
    return cos_sim

In [9]:
# Finding the most similar
B = query5
count = 0
res = []
for index, row in df.iterrows():
    cos_sim = similarity(row['embedds'],B)
    if  cos_sim > count:
        count = cos_sim
        res = [index, row['article_list'],count]
    elif count < 1.2:
        res = "Sorry parce we Couldn't find anything about that in the Constitution."
    
# print(f'Query ingresada: {B}')    
print(res)

[223, ['Los tratados, para su validez, deberán ser aprobados por el Congreso.', 'Sin embargo, el Presidente de la República podrá dar aplicación provisional a los tratados de naturaleza económica y comercial acordados en el ámbito de organismos internacionales, que así lo dispongan.', 'En este caso tan pronto como un tratado entre en vigor provisionalmente, deberá enviarse al Congreso para su aprobación.', 'Si el Congreso no lo aprueba, se suspenderá la aplicación del tratado.'], 1.3750799491259105]


In [10]:
# from fastparquet import write 
# write('Cons.parq', df)